In [1]:
%AddDeps org.bdgenomics.adam adam-core-spark2_2.11 0.24.0
%AddDeps comp.bio.aging adam-playground_2.11 0.0.12 --repository https://dl.bintray.com/comp-bio-aging/main/

Marking org.bdgenomics.adam:adam-core-spark2_2.11:0.24.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir807289334971633172/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir807289334971633172/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.jar
-> New file at /tmp/toree-tmp-dir807289334971633172/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.pom


Waiting for a Spark session to start...

Marking comp.bio.aging:adam-playground_2.11:0.0.12 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir807289334971633172/toree_add_deps/
-> https://dl.bintray.com/comp-bio-aging/main/
-> https://repo1.maven.org/maven2


In [2]:
%AddDeps org.apache.hadoop hadoop-azure 2.7.6
%AddDeps com.microsoft.azure azure-storage 2.0.0

Marking org.apache.hadoop:hadoop-azure:2.7.6 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir807289334971633172/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir807289334971633172/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.jar
-> New file at /tmp/toree-tmp-dir807289334971633172/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.pom


Waiting for a Spark session to start...

Marking com.microsoft.azure:azure-storage:2.0.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir807289334971633172/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir807289334971633172/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.jar
-> New file at /tmp/toree-tmp-dir807289334971633172/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.pom


In [3]:
import  org.apache.spark._
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._

In [4]:
def sparkHadoopConf(sc: SparkContext, acountName: String, accountKey: String) = {
  sc.hadoopConfiguration.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
  sc.hadoopConfiguration.set("fs.azure.account.key." + acountName + ".blob.core.windows.net", accountKey)
  sc
}

sparkHadoopConf: (sc: org.apache.spark.SparkContext, acountName: String, accountKey: String)org.apache.spark.SparkContext


In [5]:
def azurize(container: String, accountName: String, blobFile: String): String = "wasbs://"+container+"@"+accountName+".blob.core.windows.net/"+blobFile 

azurize: (container: String, accountName: String, blobFile: String)String


In [6]:
val account = "pipelines1"
val key = ""
val connString = s"DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=${key};EndpointSuffix=core.windows.net"
def az(path: String): String = azurize("storage", account, path)

connString = DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net
account = pipelines1
key = 


az: (path: String)String


In [7]:
def write(df: DataFrame, url: String, header: Boolean = true) = {
  df.coalesce(1).write.option("sep","\t").option("header", header).csv(url)
  url
}

write: (df: org.apache.spark.sql.DataFrame, url: String, header: Boolean)String


In [8]:
val sparkContext = sc
sparkHadoopConf(sparkContext, account, key)
  
val spark = SparkSession
  .builder()
  .appName("whales_validation")
  .getOrCreate()

sparkContext = org.apache.spark.SparkContext@40850a5c
spark = org.apache.spark.sql.SparkSession@493226c1


Spark app-20180525190630-0018: Some(http://fc9b3fce6198:4040)

In [11]:
import org.apache.spark.sql.functions._
import spark.implicits._

val toDouble = udf[Double, String]( _.toDouble)

toDouble = UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))


UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))

In [12]:
def readTSV(path: String, header: Boolean = false, sep: String = "\t", infer: Boolean = true): DataFrame = spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("inferSchema", infer)
    .option("header", header)
    .option("maxColumns", 150000)
    .csv(path)

readTSV: (path: String, header: Boolean, sep: String, infer: Boolean)org.apache.spark.sql.DataFrame
readTypedTSV: [T <: Product](path: String, header: Boolean, sep: String)(implicit tag: reflect.runtime.universe.TypeTag[T])org.apache.spark.sql.Dataset[T]


In [9]:
def readTypedTSV[T <: Product](path: String, header: Boolean = false, sep: String = "\t")
                              (implicit tag: TypeTag[T]): Dataset[T] = {
  implicit val encoder: StructType = Encoders.product[T](tag).schema
  spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("header", header)
    .schema(encoder)
    .csv(path).as[T]
}

readTypedTSV: [T <: Product](path: String, header: Boolean, sep: String)(implicit tag: reflect.runtime.universe.TypeTag[T])org.apache.spark.sql.Dataset[T]


In [13]:
val expressionsPath = "expressions"
val byGoPath = expressionsPath + "/go"
val comparison = byGoPath + "/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"
val grouped = byGoPath + "/grouped"
val ranked = byGoPath + "/grouped/ranked"


expressionsPath = expressions
byGoPath = expressions/go
comparison = expressions/go/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv
grouped = expressions/go/grouped
ranked = expressions/go/grouped/ranked


expressions/go/grouped/ranked

In [16]:
val goTable = readTSV(az(comparison), true)

goTable = [go: string, namespace: string ... 27 more fields]


[go: string, namespace: string ... 27 more fields]

In [18]:
%%dataframe goTable

go,namespace,label,gray_whale_avg,uniref90_count,taxons_count,transcripts_count,human_transcripts_count,mouse_transcripts_count,cow_transcripts_count,type,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
GO:0070062,cellular_component,extracellular exosome,95231.76295,726,929,876,2714,116,2162,complete;5prime_partial;internal;3prime_partial,82627.97828900004,43406.81879601012,103879.82613499998,189153.98861699994,34179.85539700001,63059.86755299999,12146.704848999996,407115.5446490002,402875.4094160001,107835.54761100012,12291.983165,139226.62839100015,251177.67484400002,17317.253403999995,8189.928592999998,3009.2303389999997,284041.33320799994,274118.5267729999
GO:0016021,cellular_component,integral component of membrane,81241.0623675,1391,565,1713,9561,8900,4715,5prime_partial;complete;3prime_partial;internal,62374.28512099997,27353.04877027518,55178.75498600001,108836.22230300008,11032.347553,168821.304916,28909.156348999997,198371.61491800006,189068.7378029999,100107.839614,24343.312661999982,94079.22781100006,149277.02259700003,30508.606157000006,241562.48318399998,51496.39662599999,315467.63018200017,373243.68564099993
GO:0003735,molecular_function,structural constituent of ribosome,61936.161357499994,98,380,107,467,306,261,complete;5prime_partial;3prime_partial;internal,62265.27571699998,21665.888836018064,62369.073224,103095.90135300001,5244.261549,24249.30922799999,7137.034891999996,89084.343089,87166.34631000001,61607.046997999976,8349.420524000001,88459.54913499996,70943.44369100004,4565.115427999999,50372.43014700001,11549.923136999998,94610.67463199992,72399.43132700001
GO:0005829,cellular_component,cytosol,61398.1790675,1109,994,1376,6775,5043,2348,complete;3prime_partial;5prime_partial;internal,50912.655819000014,29665.787975965563,72109.60757099999,188757.6132710001,20623.12592999999,148373.97883200005,100424.52617,86184.9085600001,90951.61626199998,71883.70231600002,12335.804511999993,124936.20551199997,132241.5113150001,13435.613476000004,105758.17169699995,43636.518504000014,111689.96638100002,102744.94863500002
GO:0005840,cellular_component,ribosome,59087.688684,89,316,99,297,123,40,complete;5prime_partial;internal;3prime_partial,58631.94472899998,23801.48132475636,65881.81530099995,103165.58999999998,2209.9476119999995,5478.616020999998,1831.119411,5936.298134999999,6073.805264,59543.432639000006,9270.050664999997,92530.64897999998,71492.44906799999,1924.4887119999999,10029.277271,2800.6401849999993,5875.225427000001,4481.681578999999
GO:0006412,biological_process,translation,58622.12868799999,105,377,116,468,286,193,complete;5prime_partial;3prime_partial;internal,58727.85268499997,20034.802301962056,58984.534049,99321.06117499995,5245.888943999999,19875.159683999995,5806.578341,66453.233819,66083.20671399997,58516.40469099998,7580.873805999998,83876.08076099999,68055.43613499998,4572.8019749999985,40136.835409,9191.198294,69545.42429499998,53565.293888999986
GO:0005634,cellular_component,nucleus,57345.87068550001,1431,2115,1830,7301,6496,2778,complete;5prime_partial;3prime_partial;internal,55747.40112099999,30295.434521280793,70441.20552599998,140240.092983,10470.131235999997,94231.85965399996,84031.996854,126193.50819700002,143407.437234,58944.34024999998,11416.312302999997,102915.768411,90533.17986400002,8808.929934000003,81393.57837199996,30863.47935199999,118101.15175700001,99420.20529599997
GO:0003723,molecular_function,RNA binding,54252.262738,621,368,821,2148,1069,1008,complete;5prime_partial;3prime_partial;internal,48843.76697,23928.740201085533,56334.43345600005,91851.60494800007,6522.695184999994,14026.664751000004,4708.064785000002,74843.61990199998,82199.186156,59660.75850600002,10025.436343999996,93364.80089000006,79318.97048099998,6672.883201000003,24

In [19]:
val components = goTable.where("namespace = 'cellular_component'")
val processes = goTable.where("namespace = 'biological_process'")
val functions = goTable.where("namespace = 'molecular_function'")

components = [go: string, namespace: string ... 27 more fields]
processes = [go: string, namespace: string ... 27 more fields]
functions = [go: string, namespace: string ... 27 more fields]


[go: string, namespace: string ... 27 more fields]

In [20]:
(components.count, processes.count, functions.count, goTable.count)

(1739,12369,4396,18597)

In [22]:
write(components, az(grouped + "/by_cellular_component.tsv"))


lastException: Throwable = null


wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/grouped/by_cellular_component.tsv

In [23]:
write(processes, az(grouped + "/by_biological_process.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/grouped/by_biological_process.tsv

In [24]:
write(functions, az(grouped + "/by_molecular_function.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/grouped/by_molecular_function.tsv

In [12]:
import org.apache.spark.sql.{ColumnName, Dataset}
import org.apache.spark.sql.expressions.Window

def rank(df: DataFrame, name: String, rankSuffix: String = "_rank") =
  df.withColumn(name + rankSuffix, org.apache.spark.sql.functions.dense_rank().over(Window.orderBy(new ColumnName(name).desc)))

def ranks(df: DataFrame,
          names: Seq[String],
          rankSuffix: String = "_rank") = names.foldLeft(df){
  case (f, n)=> rank(f, n, rankSuffix)
}

rank: (df: org.apache.spark.sql.DataFrame, name: String, rankSuffix: String)org.apache.spark.sql.DataFrame
ranks: (df: org.apache.spark.sql.DataFrame, names: Seq[String], rankSuffix: String)org.apache.spark.sql.DataFrame


In [1]:
val rankedCols = components.columns.drop(11)++List("gray_whale_avg")

rankedCols = Array(gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney, gray_whale_avg)


[gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney, gray_whale_avg]

In [2]:
val rankedComponents = ranks(components, rankedCols)
val rankedProcesses = ranks(processes, rankedCols)
val rankedFunctions = ranks(functions, rankedCols)

rankedComponents = [go: string, namespace: string ... 46 more fields]
rankedProcesses = [go: string, namespace: string ... 46 more fields]
rankedFunctions = [go: string, namespace: string ... 46 more fields]


[go: string, namespace: string ... 46 more fields]

In [3]:
write(rankedComponents, az(grouped + "/by_cellular_component_ranked.tsv"))
write(rankedProcesses, az(grouped + "/by_biological_process_ranked.tsv"))
write(rankedFunctions, az(grouped + "/by_molecular_function_ranked.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/grouped/by_molecular_function_ranked.tsv

In [26]:
%%dataframe rankedComponents

go,namespace,label,gray_whale_avg,uniref90_count,taxons_count,transcripts_count,human_transcripts_count,mouse_transcripts_count,cow_transcripts_count,type,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney,gray_whale_liver_rank,bowhead_whale_liver_rank,minke_liver_rank,NMR_liver_rank,human_liver_rank,mouse_totalRNA_liver_rank,mouse_mRNA_liver_rank,cow_totalRNA_liver_rank,cow_mRNA_liver_rank,gray_whale_kidney_rank,bowhead_whale_kidney_rank,minke_kidney_rank,NMR_kidney_rank,human_kidney_rank,mouse_totalRNA_kidney_rank,mouse_mRNA_kidney_rank,cow_totalRNA_kidney_rank,cow_mRNA_kidney_rank,gray_whale_avg_rank
GO:0070062,cellular_component,extracellular exosome,95231.76295,726,929,876,2714,116,2162,complete;5prime_partial;internal;3prime_partial,82627.97828900004,43406.81879601012,103879.82613499998,189153.98861699994,34179.85539700001,63059.86755299999,12146.704848999996,407115.5446490002,402875.4094160001,107835.54761100012,12291.983165,139226.62839100015,251177.67484400002,17317.253403999995,8189.928592999998,3009.2303389999997,284041.33320799994,274118.5267729999,1,1,1,1,1,12,16,1,1,1,4,1,1,3,46,45,2,2,1
GO:0016021,cellular_component,integral component of membrane,81241.0623675,1391,565,1713,9561,8900,4715,5prime_partial;complete;3prime_partial;internal,62374.28512099997,27353.04877027518,55178.75498600001,108836.22230300008,11032.347553,168821.304916,28909.156348999997,198371.61491800006,189068.7378029999,100107.839614,24343.312661999982,94079.22781100006,149277.02259700003,30508.606157000006,241562.48318399998,51496.39662599999,315467.63018200017,373243.68564099993,2,4,6,6,8,3,8,2,2,2,1,5,2,1,2,2,1,1,2
GO:0005829,cellular_component,cytosol,61398.1790675,1109,994,1376,6775,5043,2348,complete;3prime_partial;5prime_partial;internal,50912.655819000014,29665.787975965563,72109.60757099999,188757.6132710001,20623.12592999999,148373.97883200005,100424.52617,86184.9085600001,90951.61626199998,71883.70231600002,12335.804511999993,124936.20551199997,132241.5113150001,13435.613476000004,105758.17169699995,43636.518504000014,111689.96638100002,102744.94863500002,5,3,2,2,4,5,2,5,6,3,3,2,3,4,3,3,4,4,3
GO:0005840,cellular_component,ribosome,59087.688684,89,316,99,297,123,40,complete;5prime_partial;internal;3prime_partial,58631.94472899998,23801.48132475636,65881.81530099995,103165.58999999998,2209.9476119999995,5478.616020999998,1831.119411,5936.298134999999,6073.805264,59543.432639000006,9270.050664999997,92530.64897999998,71492.44906799999,1924.4887119999999,10029.277271,2800.6401849999993,5875.225427000001,4481.681578999999,3,6,4,7,37,71,70,69,71,5,7,6,7,24,42,46,63,71,4
GO:0005634,cellular_component,nucleus,57345.87068550001,1431,2115,1830,7301,6496,2778,complete;5prime_partial;3prime_partial;internal,55747.40112099999,30295.434521280793,70441.20552599998,140240.092983,10470.131235999997,94231.85965399996,84031.996854,126193.50819700002,143407.437234,58944.34024999998,11416.312302999997,102915.768411,90533.17986400002,8808.929934000003,81393.57837199996,30863.47935199999,118101.15175700001,99420.20529599997,4,2,3,4,10,9,3,4,4,6,5,3,4,9,7,5,3,5,5
GO:0005737,cellular_component,cytoplasm,51021.87724949999,1070,1897,1342,4490,5424,2333,complete;5prime_partial;3prime_partial;internal,43367.513537999985,24393.98329442446,63289.08200100004,177635.51962800007,6716.158039999998,135388.78987699997,34234.04624200001,79546.89437599998,85249.95693999997,58676.24096099999,10090.376369000003,102810.643446,85731.96628100002,7004.031145,87788.121675,34874.35143699998,101489.413696,87811.42796300001,6,5,5,3,19,6,6,7,7,7,6,4,5,12,5,4,5,8,6
GO:0005813,cellular_component,centrosome,42454.98833149999,127,243,162,632,769,382,complete;5prime_partial;internal;3prime_partial,24852.939731,1933.4432743038858,4332.0924300000015

In [13]:
case class UniprotMapping(
                           uniprot_ac: String,
                           uniprot_id: String,
                           entrez: String,
                           refSeq: String,
                           gi: String,
                           pdb: String,
                           go: String,
                           uniref100: String,
                           uniref90: String,
                           uniref50: String,
                           uniparc: String,
                           pir: String,
                           taxon: String,
                           mim: String,
                           unigene: String,
                           pubmed: String,
                           embl: String,
                           embl_cds: String,
                           ensembl: String,
                           ensembl_trs: String,
                           ensembl_pro: String,
                           additional_pubmed: String
                         )
{
  lazy val annotations = go.split(";").map(_.trim).toSet
}

defined class UniprotMapping


In [14]:
val mapping =  readTSV[UniprotMapping](az("/indexes/uniprot/idmapping_selected.tab")).toDF(                           uniprot_ac: String,
                           uniprot_id: String,
                           entrez: String,
                           refSeq: String,
                           gi: String,
                           pdb: String,
                           go: String,
                           uniref100: String,
                           uniref90: String,
                           uniref50: String,
                           uniparc: String,
                           pir: String,
                           taxon: String,
                           mim: String,
                           unigene: String,
                           pubmed: String,
                           embl: String,
                           embl_cds: String,
                           ensembl: String,
                           ensembl_trs: String,
                           ensembl_pro: String,
                           additional_pubmed: String)

idmapping = [uniprot_ac: string, uniprot_id: string ... 20 more fields]


[uniprot_ac: string, uniprot_id: string ... 20 more fields]

In [20]:
idmapping.take(1)

[UniprotMapping(Q6GZX4,001R_FRG3G,2947773,YP_031579.1,81941549; 49237298,null,GO:0006355; GO:0046782; GO:0006351,UniRef100_Q6GZX4,UniRef90_Q6GZX4,UniRef50_Q6GZX4,UPI00003B0FD4,null,654924,null,null,15165820,AY548484,AAT09660.1,null,null,null,null)]

For Mitya
=====

Two cells below are for Mitya to get infromation about Uniref90 and Go. As

In [25]:
val go = "GO:0006355" //put your GO term inside quotes
val frame = idmapping.filter(u=>u.go.contains(go)).toDF()

go = GO:0006355
frame = [uniprot_ac: string, uniprot_id: string ... 20 more fields]


[uniprot_ac: string, uniprot_id: string ... 20 more fields]

In [27]:
%%dataframe frame

--output  The type of the output: ...


ERROR: Task not serializable
%%dataframe [arguments]
DATAFRAME_CODE

DATAFRAME_CODE can be any numbered lines of code, as long as the
last line is a reference to a variable which is a DataFrame.
    Option    Description                       
------    -----------                       
--limit   The number of records to return   
          (default: 10)                     
--output  The type of the output: html, csv,
          json (default: html)              
Option    Description                       
------    -----------                       
--limit   The number of records to return   
          (default: 10)                     
--output  The type of the output: html, csv,
          json (default: html)              
Option    Description                       
------    -----------                       
--limit   The number of records to return   
          (default: 10)                     
--output  The type of the output: html, csv,
          json (default: html)      